<a href="https://colab.research.google.com/github/albertodiez1980LaRioja/Prueba/blob/master/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# En este notebook probaremos la efectividad sobre el grupo de entrenamiento y un grupo de test externo de los diferentes algoritmos

!wget https://www.dropbox.com/s/hvq2c6gwm8hyvp1/Test.csv?dl=0 -O Test.csv
!wget https://www.dropbox.com/s/n6akt2v5xupw5ef/Train.csv?dl=0 -O Train.csv

--2020-05-06 15:42:19--  https://www.dropbox.com/s/hvq2c6gwm8hyvp1/Test.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hvq2c6gwm8hyvp1/Test.csv [following]
--2020-05-06 15:42:20--  https://www.dropbox.com/s/raw/hvq2c6gwm8hyvp1/Test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5f88bbf697aec5138cd93347a3.dl.dropboxusercontent.com/cd/0/inline/A3PzzLhicQWzVEcW7vafI9Bxds0fwKlV5h-FrVXsTr_8e0fI2v6_DXuEvkIFAvNp0GFcT6EnImNLRSYKOwBQaS-2mM2CrgHxMTJqVl8u8HXrLkp-8dYjMCt5dvQ4GKWQqsQ/file# [following]
--2020-05-06 15:42:20--  https://uc5f88bbf697aec5138cd93347a3.dl.dropboxusercontent.com/cd/0/inline/A3PzzLhicQWzVEcW7vafI9Bxds0fwKlV5h-FrVXsTr_8e0fI2v6_DXuEvkIFAvNp0GFcT6EnImNLRSYKOwBQaS-2mM2CrgHxMTJqVl8u8HXrLkp-8dYjMC

In [28]:
# ahora metemos los dataset en variables

import pandas as pd

# hay que tener presente que el primer descriptor es la ruta y nombre del archivo
# el segundo es la epoca y el tercero es el tipo
Testdf = pd.read_csv('Test.csv',header=None)
Testdf.head()

Traindf=pd.read_csv('Train.csv',header=None)
Traindf.head()



,0,1,2
0,piedras/5/6086.jpg,5,piedras
1,piedras/5/6965.jpg,5,piedras
2,piedras/4/7006.jpg,4,piedras
3,piedras/4/6941.jpg,4,piedras
4,piedras/4/6869.jpg,4,piedras


In [29]:
# ahora toca cargar los modelos
# Carga de librerias
from fastai.vision import *
import os
from google.colab import files

!wget https://www.dropbox.com/s/p5ufjrl6dgeo75d/MODELOS.zip?dl=0 -O modelos.zip --quiet
!unzip -qq modelos.zip 

learnBruta = load_learner("MODELOS/FuerzaBruta")
learnGeneral = load_learner("MODELOS/General")
learnCrucesPortavelasCampanas = load_learner("MODELOS/crucesPortavelasCampanas")
learnEnganches = load_learner("MODELOS/enganches")
learnEstatuas = load_learner("MODELOS/estatuas")
learnPesosMonedas = load_learner("MODELOS/pesosMonedas")
learnPiedras = load_learner("MODELOS/pesosMonedas")
learnPinturas = load_learner("MODELOS/Pinturas")
learnRecipientes = load_learner("MODELOS/recipientes")
learnRopatelas = load_learner("MODELOS/ropaTelas")
learnUtensiliosAgricultura = load_learner("MODELOS/utensiliosAgricultura")

replace MODELOS/.directory? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [30]:
!wget https://www.dropbox.com/s/f3qaaadn8axihb3/CLASIFICADOS_2produccion.zip?dl=0 -O CLASIFICADOS_2produccion.zip
!unzip CLASIFICADOS_2produccion.zip


--2020-05-06 15:44:51--  https://www.dropbox.com/s/f3qaaadn8axihb3/CLASIFICADOS_2produccion.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/f3qaaadn8axihb3/CLASIFICADOS_2produccion.zip [following]
--2020-05-06 15:44:51--  https://www.dropbox.com/s/raw/f3qaaadn8axihb3/CLASIFICADOS_2produccion.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd4cc22b3e0d1020fc27b40ed39.dl.dropboxusercontent.com/cd/0/inline/A3MPjH1FPa8MAG-85MPVQK7DDsvfjcmOlnjzG-t_hzg63RTXbdVSHOWY11iexR4o79f9LkU_lwngS-Uu9_v6H5KxU7OL8z3Y7j-m6zHOUWqRWcp27pDbiK3Mz_h9NTXt4As/file# [following]
--2020-05-06 15:44:52--  https://ucd4cc22b3e0d1020fc27b40ed39.dl.dropboxusercontent.com/cd/0/inline/A3MPjH1FPa8MAG-85MPVQK7DDsvfjcmOlnjzG-t_hzg63RTXbdVSHOWY11i

In [31]:
Traindf[0][0]

'piedras/5/6086.jpg'

In [32]:
# ahora testeamos los aciertos y equivocaciones de los algoritmos en los datasets

import warnings
warnings.filterwarnings('ignore')

def testeaDFepoca(df,modelo,directorio):
  acertados=0
  testeados=0
  for index, row in df.iterrows():    
    if not('.directory' in str(row[0])):
      pred_class,pred_idx,outputs=modelo.predict(open_image(directorio+row[0]))
      if(str(pred_class)==str(row[1])):
        acertados+=1
      if(testeados % 50 ==0):
        print("Espera un poco mas")
      testeados+=1
  print("Acertados: "+str(acertados)+ " fallados: "+str(testeados-acertados)+
        " accuracy: "+str(acertados/(testeados)))

testeaDFepoca(Traindf,learnBruta,'CLASIFICADOS_2produccion/')


Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Acertados: 575 fallados: 35 accuracy: 0.9426229508196722


In [33]:
!wget https://www.dropbox.com/s/64ags91s00oplpt/ImagenesTest.zip?dl=0 -O ImagenesTest.zip
!unzip ImagenesTest.zip

--2020-05-06 15:46:48--  https://www.dropbox.com/s/64ags91s00oplpt/ImagenesTest.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/64ags91s00oplpt/ImagenesTest.zip [following]
--2020-05-06 15:46:48--  https://www.dropbox.com/s/raw/64ags91s00oplpt/ImagenesTest.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2f3407dd8f0cd27d091d037720.dl.dropboxusercontent.com/cd/0/inline/A3NX7WYjCoIIYbRsZQNq1JeB2ty_FjXr8bf2DNJ609uh7xhMOagulgPEqmMk4IRGLaeT37OzM_fWc0qkU8uiy3a378ZhlPoPiQJCvsQXtjtCjWbpu_2xXmzQx3ob6T_jM38/file# [following]
--2020-05-06 15:46:48--  https://uc2f3407dd8f0cd27d091d037720.dl.dropboxusercontent.com/cd/0/inline/A3NX7WYjCoIIYbRsZQNq1JeB2ty_FjXr8bf2DNJ609uh7xhMOagulgPEqmMk4IRGLaeT37OzM_fWc0qkU8uiy3a378ZhlPo

In [34]:
testeaDFepoca(Testdf,learnBruta,'ImagenesTest/')
# vemos que la efectividad ha bajado mucho, la naturaleza del problema es excesivamente complicada
# teniendo en cuenta esto el resultado no ha sido tan malo

Espera un poco mas
Espera un poco mas
Acertados: 29 fallados: 26 accuracy: 0.5272727272727272


In [41]:
def testeaDFtipo(df,modelo,directorio):
  acertados=0
  testeados=0
  for index, row in df.iterrows(): 
    if not('.directory' in str(row[0])):
      pred_class,pred_idx,outputs=modelo.predict(open_image(directorio+row[0]))
      if(str(pred_class)==str(row[2])): # lo comparamos con el tipo
        acertados+=1
      if(testeados % 50 ==0):
        print("Espera un poco mas")
      testeados+=1
  print("Acertados: "+str(acertados)+ " fallados: "+str(testeados-acertados)+
        " accuracy: "+str(acertados/(testeados)))

testeaDFtipo(Testdf,learnGeneral,'ImagenesTest/')
# no sale mal el tipo

Espera un poco mas
Espera un poco mas
Acertados: 39 fallados: 16 accuracy: 0.7090909090909091


In [42]:
testeaDFtipo(Traindf,learnGeneral,'CLASIFICADOS_2produccion/')
# sale muy bien con el entrenamiento, tal vez sospechosamente alto

Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Espera un poco mas
Acertados: 592 fallados: 18 accuracy: 0.9704918032786886


In [43]:
def testeaDFedad_mediante_tipo(df,modelo,directorio):
  acertados=0
  testeados=0
  for index, row in df.iterrows(): 
    if not('.directory' in str(row[0])):
      pred_class,pred_idx,outputs=modelo.predict(open_image(directorio+row[0]))
      if(str(pred_class)==str(row[2])): # lo comparamos con el tipo
        if str(pred_class)=='crucesPortavelasCampanas':
          pred_class,pred_idx,outputs=learnCrucesPortavelasCampanas.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='enganches':
          pred_class,pred_idx,outputs=learnEnganches.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='estatuas':
          pred_class,pred_idx,outputs=learnEstatuas.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='pesosMonedas':
          pred_class,pred_idx,outputs=learnPesosMonedas.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='piedras':
          pred_class,pred_idx,outputs=learnPiedras.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='pinturas':
          pred_class,pred_idx,outputs=learnPinturas.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='recipientes':
          pred_class,pred_idx,outputs=learnRecipientes.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='ropaTelas':
          pred_class,pred_idx,outputs=learnRopatelas.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
        if str(pred_class)=='utensiliosAgricultura':
          pred_class,pred_idx,outputs=learnUtensiliosAgricultura.predict(open_image(directorio+row[0]))
          if(str(pred_class)==str(row[1])):
            acertados+=1
      if(testeados % 50 ==0):
        print("Espera un poco mas")
      testeados+=1
  print("Acertados: "+str(acertados)+ " fallados: "+str(testeados-acertados)+
        " accuracy: "+str(acertados/(testeados)))
  
testeaDFedad_mediante_tipo(Testdf,learnGeneral,'ImagenesTest/')  


Espera un poco mas
Espera un poco mas
Acertados: 9 fallados: 46 accuracy: 0.16363636363636364
